In [64]:
import pandas as pd
import numpy as np
import sqlalchemy
from tqdm import tqdm
tqdm.pandas()

In [65]:
creds = {
  "drivername": "postgresql+psycopg2",
  "username": "postgres",
  "password": "postgres1",
  "host": "84.201.161.167",
  "port": "5432",
  "database": "postgres"
}
engine = sqlalchemy.create_engine(sqlalchemy.engine.URL.create(**creds))
obj = pd.read_sql('''
SELECT col_754 as "Нет техпаспорта МосгорБТИ",
    col_758_id as "Серия проекта",
    col_769_id as "Материал стен",
    col_770_id as "Признак аварийности здания",
    col_775_id as "Очередность уборки кровли",
    col_781_id as "Материал кровли",
    col_2463_id as "Тип жилищного фонда",
    col_3163_id as "Статус МКД",
    col_3243_id as "Статус управления МКД",
    col_756 as "Год постройки",
    col_759 as "Количество этажей",
    col_760 as "Количество подъездов",
    col_761 as "Количество квартир",
    col_762 as "Общая площадь",
    col_763 as "Общая площадь жилых помещений",
    col_764 as "Общая площадь нежилых помещений",
    col_771 as "Количество пассажирских лифтов",
    col_772 as "Количество грузопассажирских лифтов",
    col_3363 as "Количество грузовых лифтов",
    col_782 as "UNOM"
FROM api_object
''', engine)
obj.rename(columns={'Количество грузопассажирских лиф':'Количество грузопассажирских лифтов'}, inplace=True)
obj.set_index('UNOM', inplace=True)
obj['Нет техпаспорта МосгорБТИ'].replace('.ет.*БТИ', 'Нет техпаспорта МосгорБТИ', regex=True, inplace=True)
obj['Нет техпаспорта МосгорБТИ'].fillna(False, inplace=True)
obj['Серия проекта'].fillna(2048743, inplace=True)
obj['Материал стен'].fillna(101812168, inplace=True)
obj['Признак аварийности здания'].fillna(22728487, inplace=True)
obj['Материал кровли'].fillna(22289209, inplace=True)
obj.fillna(0, inplace=True)
obj['Нет техпаспорта МосгорБТИ'] = obj['Нет техпаспорта МосгорБТИ'].astype(bool)
obj=obj.astype(int)
obj = obj.astype({'Нет техпаспорта МосгорБТИ': 'category',
                          'Серия проекта': 'category',
                          'Материал стен': 'category',
                          'Признак аварийности здания': 'category',
                          'Очередность уборки кровли': 'category',
                          'Материал кровли': 'category',
                          'Тип жилищного фонда': 'category',
                          'Статус МКД': 'category',
                          'Статус управления МКД': 'category'})

In [66]:
obj.sample()

,Нет техпаспорта МосгорБТИ,Серия проекта,Материал стен,Признак аварийности здания,Очередность уборки кровли,Материал кровли,Тип жилищного фонда,Статус МКД,Статус управления МКД,Год постройки,Количество этажей,Количество подъездов,Количество квартир,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Количество грузовых лифтов
UNOM,,,,,,,,,,,,,,,,,,,
2405146,1,2048912,2048929,22728487,0,22289209,0,68481453,0,0,1,0,0,0,0,0,0,0,0


In [67]:
inced = pd.read_sql('''
SELECT unom, name, system
FROM api_problem
JOIN api_problemtype ON problem_type_id = local_id
''', engine)
inced.rename(columns={'name':'Наименование', 'system': 'Источник'}, inplace=True)

In [68]:
inced.sample(3)

,unom,Наименование,Источник
941087,27719,Изменение конфигурации УСПД,ASUPR
163662,5271685,Засор мусоропровода,EDC
309769,1440,Критичное отклонение температуры ГВС ниже норм...,ASUPR


In [104]:
#Крыша
# inced_weight = {
#     'Аварийная протечка с кровли': 100,
#     'Протечка с кровли': 70,
#     'Неочищенная кровля': 10,
#     'Повреждение кровли': 20
# }

#Мусор
# inced_weight = {
# 'Загрязнение/замусоренность козырька': 10,
# 'Загрязнение/замусоренность подвала/полуподвала': 10,
# 'Загрязнение/замусоренность подъезда': 10,
# 'Загрязнение/замусоренность территории': 10,
# 'Засор мусоропровода': 50,
# 'Неудовлетворительное санитарное содержание мусоропровода': 10,
# 'Неудовлетворительное техническое содержание мусоропровода': 10
# }

#Электрика
# inced_weight = {
# 'Отсутствие освещения в местах общего пользования': 20,
# 'Повреждение системы электропроводки/щитового оборудования': 60,
# 'Повреждение/поломка светильника': 10,
# 'Поломка освещения перед подъездом': 30
# }

#Лифт
# inced_weight = {
# 'Застревание в лифте': 30,
# 'Лифт требует ремонта': 70,
# 'Отсутствие освещения в лифте': 10,
# 'Поломка лифта': 70
# }

#ГВС
# inced_weight = {
# 'Критичное отклонение температуры ГВС ниже нормы днем (мониторинг)': 45,
# 'Недогрев ГВС': 20,
# 'Отклонение ГВС ниже нормы днем (мониторинг)': 20,
# 'Отклонение ГВС ниже нормы ночью (мониторинг)': 20,
# 'Отрицательные интегральные значения': 10,
# 'Отсутствие ГВС в доме': 70,
# 'Отсутствует циркуляция ГВС': 45,
# 'Превышение температуры подачи ЦО (Перетоп)': 20
# }

#Подъезд
inced_weight = {
'Аварийное повреждение лестницы': 60,
'Аварийное повреждение лестницы': 60,
'Блокировка входной двери': 30,
'Загрязнение входной двери': 20,
'Загрязнение подъемной платформы для инвалидов': 20,
'Загрязнение/замусоренность подъезда': 20,
'Неработоспособность подъемной платформы для инвалидов': 30,
'Повреждение козырька подъезда': 20,
'Повреждение подъемной платформы для инвалидов': 30,
'Повреждение элементов входной двери': 20
}

In [105]:
inced_roof = inced.query(f'Наименование in {list(inced_weight.keys())}')
inced_roof.set_index('unom', drop=True, inplace=True)
inced_roof['counter'] = 1
inced_roof.sample(3)

<ipython-input-105-f96518e4489c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inced_roof['counter'] = 1


,Наименование,Источник,counter
unom,,,
20148,Загрязнение/замусоренность подъезда,EDC,1
26823,Блокировка входной двери,EVAGD,1
371,Повреждение элементов входной двери,NG,1


In [106]:
groupped_inced_roof = inced_roof.groupby([inced_roof.index,'Наименование', 'Источник']).count()
groupped_inced_roof.reset_index(inplace=True)
groupped_inced_roof.rename(columns={'level_0': 'unom', 'counter': 'Кол-во обращений'}, inplace=True)
groupped_inced_roof.set_index('unom', inplace=True)
groupped_inced_roof

,Наименование,Источник,Кол-во обращений
unom,,,
308,Загрязнение/замусоренность подъезда,EDC,3
308,Повреждение элементов входной двери,NG,5
309,Загрязнение/замусоренность подъезда,EDC,6
309,Повреждение элементов входной двери,NG,7
310,Загрязнение/замусоренность подъезда,EDC,5
...,...,...,...
5271685,Загрязнение/замусоренность подъезда,EDC,114
5271685,Повреждение элементов входной двери,NG,44
5273743,Загрязнение/замусоренность подъезда,EDC,9


In [107]:
df = groupped_inced_roof.merge(obj['Количество квартир'], how='left', left_index=True, right_index=True)
df.head(3)

,Наименование,Источник,Кол-во обращений,Количество квартир
308,Загрязнение/замусоренность подъезда,EDC,3,10
308,Повреждение элементов входной двери,NG,5,10
309,Загрязнение/замусоренность подъезда,EDC,6,24


In [108]:
df['Вес'] = df.progress_apply(lambda row: inced_weight[row['Наименование']], axis=1)
df.sample(3)

100%|███████████████████████████████████| 9871/9871 [00:00<00:00, 17373.44it/s]


,Наименование,Источник,Кол-во обращений,Количество квартир,Вес
18898,Повреждение элементов входной двери,NG,8,207,20
17665,Загрязнение/замусоренность подъезда,EDC,1,12,20
22844,Аварийное повреждение лестницы,EVAGD,1,108,60


In [109]:
df['Балл'] = round(df['Кол-во обращений']*df['Вес']/(df['Количество квартир']+1), 2)
df.head(3)

,Наименование,Источник,Кол-во обращений,Количество квартир,Вес,Балл
308,Загрязнение/замусоренность подъезда,EDC,3,10,20,5.45
308,Повреждение элементов входной двери,NG,5,10,20,9.09
309,Загрязнение/замусоренность подъезда,EDC,6,24,20,4.80


In [127]:
mkd_rating = df.groupby(df.index)['Балл'].transform('sum')
mkd_rating.name = 'Рейтинг дома'
res = (df.merge(mkd_rating, how='left', left_index=True, right_index=True)
  .rename_axis('unom')
  .sort_values(by = ['Рейтинг дома', 'unom', 'Наименование', 'Источник'], ascending = [False, True, True, True]))
res = res.drop_duplicates()
res

,Наименование,Источник,Кол-во обращений,Количество квартир,Вес,Балл,Рейтинг дома
unom,,,,,,,
5271685,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3240.00
5271685,Загрязнение входной двери,NG,1,0,20,20.00,3240.00
5271685,Загрязнение/замусоренность подъезда,EDC,114,0,20,2280.00,3240.00
5271685,Повреждение элементов входной двери,NG,44,0,20,880.00,3240.00
5012968,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3230.00
...,...,...,...,...,...,...,...
240007,Повреждение элементов входной двери,NG,1,377,20,0.05,0.05
240149,Повреждение элементов входной двери,NG,1,413,20,0.05,0.05
64128,Повреждение элементов входной двери,NG,1,471,20,0.04,0.04


In [128]:
# Установить количество домов, которые планируется ремонтировать
quantity_to_repair = 300
res = res[:quantity_to_repair]
res

,Наименование,Источник,Кол-во обращений,Количество квартир,Вес,Балл,Рейтинг дома
unom,,,,,,,
5271685,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3240.00
5271685,Загрязнение входной двери,NG,1,0,20,20.00,3240.00
5271685,Загрязнение/замусоренность подъезда,EDC,114,0,20,2280.00,3240.00
5271685,Повреждение элементов входной двери,NG,44,0,20,880.00,3240.00
5012968,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3230.00
...,...,...,...,...,...,...,...
18766,Загрязнение/замусоренность подъезда,EDC,105,206,20,10.14,12.37
18766,Повреждение козырька подъезда,EDC,1,206,20,0.10,12.37
18766,Повреждение элементов входной двери,NG,22,206,20,2.13,12.37


In [129]:
res['Месяц'] = pd.cut(res['Рейтинг дома'], bins=12, labels=range(12,0,-1))
res

<ipython-input-129-2426dce91ce9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Месяц'] = pd.cut(res['Рейтинг дома'], bins=12, labels=range(12,0,-1))


,Наименование,Источник,Кол-во обращений,Количество квартир,Вес,Балл,Рейтинг дома,Месяц
unom,,,,,,,,
5271685,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3240.00,1
5271685,Загрязнение входной двери,NG,1,0,20,20.00,3240.00,1
5271685,Загрязнение/замусоренность подъезда,EDC,114,0,20,2280.00,3240.00,1
5271685,Повреждение элементов входной двери,NG,44,0,20,880.00,3240.00,1
5012968,Аварийное повреждение лестницы,EVAGD,1,0,60,60.00,3230.00,1
...,...,...,...,...,...,...,...,...
18766,Загрязнение/замусоренность подъезда,EDC,105,206,20,10.14,12.37,12
18766,Повреждение козырька подъезда,EDC,1,206,20,0.10,12.37,12
18766,Повреждение элементов входной двери,NG,22,206,20,2.13,12.37,12


In [111]:
res.to_excel('maintenance_entrance.xlsx')

<!-- ## План по датам -->

In [15]:
# cap_rem = pd.read_excel('3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx')

In [41]:
# cap_rem['PLAN_DATE_START'] = pd.to_datetime(cap_rem['PLAN_DATE_START'])
# cap_rem['PLAN_DATE_END'] = pd.to_datetime(cap_rem['PLAN_DATE_END'])
# cap_rem['FACT_DATE_START'] = pd.to_datetime(cap_rem['FACT_DATE_START'])
# cap_rem['FACT_DATE_END'] = pd.to_datetime(cap_rem['FACT_DATE_END'])
# cap_rem['PLAN_DATE_START_m'] = cap_rem['PLAN_DATE_START'].dt.month
# cap_rem['PLAN_DATE_END_m'] = cap_rem['PLAN_DATE_END'].dt.month
# cap_rem['FACT_DATE_START_m'] = cap_rem['FACT_DATE_START'].dt.month
# cap_rem['FACT_DATE_END_m'] = cap_rem['FACT_DATE_END'].dt.month

In [44]:
# cap_rem.query('WORK_NAME == "ремонт крыши"').set_index('PLAN_DATE_START').groupby(pd.Grouper(freq='M'))['global_id'].count()

PLAN_DATE_START
2022-01-31      3
2022-02-28      0
2022-03-31      0
2022-04-30      3
2022-05-31      3
2022-06-30      1
2022-07-31      1
2022-08-31      0
2022-09-30      0
2022-10-31      0
2022-11-30    132
Freq: M, Name: global_id, dtype: int64

In [62]:
# cap_rem.query('WORK_NAME == "ремонт крыши"').groupby('PLAN_DATE_START_m')['global_id'].count()

PLAN_DATE_START_m
1       3
4       3
5       3
6       1
7       1
11    132
Name: global_id, dtype: int64

In [60]:
# cap_rem

,global_id,PERIOD,WORK_NAME,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END,AdmArea,District,Address,UNOM,PLAN_DATE_START_m,PLAN_DATE_END_m,FACT_DATE_START_m,FACT_DATE_END_m
0,351060,2022,замена лифтового оборудования,1.0,1,2022-07-15,2022-08-28,2022-06-27,2022-10-14,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",20151,7,8,6,10
1,430496,2022,замена лифтового оборудования,1.0,1,2022-09-30,2022-10-14,2022-09-30,2022-10-21,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",20151,9,10,9,10
2,348580,2022,ремонт внутридомовых инженерных систем водоотв...,NaN,NaN,2022-02-21,2022-06-20,2022-01-02,2022-05-25,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466,2,6,1,5
3,348581,2022,ремонт внутридомовых инженерных систем теплосн...,NaN,NaN,2022-04-21,2022-10-07,2022-07-05,2022-03-08,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466,4,10,7,3
4,348578,2022,ремонт внутридомовых инженерных систем холодно...,NaN,NaN,2022-02-21,2022-06-20,2022-01-02,2022-06-27,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466,2,6,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742,366525,2022,ремонт внутридомовых инженерных систем теплосн...,NaN,NaN,2022-04-21,2022-10-07,2022-07-03,2022-11-08,Восточный административный округ,район Гольяново,"Российская Федерация, город Москва, внутригоро...",9177,4,10,7,11
1743,366526,2022,"ремонт подвальных помещений, относящихся к общ...",NaN,NaN,2022-05-21,2022-09-30,2022-04-02,2022-06-17,Восточный административный округ,район Гольяново,"Российская Федерация, город Москва, внутригоро...",9177,5,9,4,6
1744,366523,2022,ремонт внутридомовых инженерных систем водоотв...,NaN,NaN,2022-01-02,2022-05-31,2022-04-02,2022-04-08,Восточный административный округ,район Гольяново,"Российская Федерация, город Москва, внутригоро...",9177,1,5,4,4
1745,394724,2022,ремонт внутреннего водостока,NaN,NaN,2022-11-11,2022-12-31,2022-10-10,2022-11-30,Восточный административный округ,район Ивановское,"Российская Федерация, город Москва, внутригоро...",12178,11,12,10,11
